# Text Deploy


Using the `move_draft_text` script is simple enough. I find though that many deploys require some stage of manual coding. Other cases, such as multi-text deploys, require running the script lots of times, which can be a drain on an engineers time. This notebook is a collection of code I've written that helps make these deploys a lot easier.


In [ ]:
import os
import sys
import django

SEFARIA_PROJECT_PATH = os.environ['SEFARIA_PROJECT_PATH']




I like to use an environment variable to set the script path.

In [ ]:
if SEFARIA_PROJECT_PATH not in sys.path:
    sys.path.append(SEFARIA_PROJECT_PATH)
sys.path.append(f'{SEFARIA_PROJECT_PATH}/scripts')

When running from the root of the Sefaria-Project directory, this logic can be simplified to:

In [ ]:
sys.path.append('./scripts')

In [ ]:
django.setup()
from sefaria.model import *
from move_draft_text import ServerTextCopier

Set the destination as needed. As before, I recommend reading the apikey from the environment, but to each his/her own. These values need to be set in order to initialize a `ServerTextCopier` but do not technically need to be _correct_ if an api call is not being made.

In [ ]:
dest = 'https://www.sefaria.org'
apikey = os.environ['APIKEY']

We now need some titles for the deploy.

In [ ]:
titles = library.get_indexes_in_category('Abarbanel')

If the criteria for uploading texts is complex, we can do this in two steps by loading up full indices and then filtering them on some criteria, e.g.

In [ ]:
indices = library.get_indexes_in_category("Abarbanel", include_dependant=True, full_records=True)
titles = [i.title for i in indices if 'Prophets' in i.categories]

In [ ]:
def upload_indices(text_title):
    copier = ServerTextCopier(dest, apikey, text_title, post_index=True,)
    copier.do_copy()
    
def upload_versions(text_title):
    copier = ServerTextCopier(dest, apikey, text_title, post_index=False, versions='all')
    copier.do_copy()
    
def upload_links(text_title):
    copier = ServerTextCopier(dest, apikey, text_title, post_index=False, post_links=2)
    copier.do_copy()

I'm going to leave it to the developer to iterate over the titles and perform the necessary upload

In [ ]:
for t in titles:
    pass

Here we will aggregate our links and then dump them to a json file. These can then be imported via the backend on another server

In [ ]:
import json
links = []
export_filename = 'links_json.json'  # feel free to change this
for t in titles:
    copier = ServerTextCopier(dest, apikey, t, post_index=False, post_links=2)
    copier.load_objects()
    links += [l.contents() for l in copier._linkset if not getattr(l, 'source_text_oid', None)]
with open(export_filename, 'w') as fp:
    json.dump(links, fp)

This is code we can use on the destination server to import a links json file

In [ ]:
import json
from sefaria import tracker
from sefaria.system.exceptions import DuplicateRecordError
from sefaria.local_settings import USE_VARNISH
from sefaria.system.varnish.wrapper import invalidate_ref

In [ ]:
import_filename = 'foo'
with open(import_filenam) as fp:
    links = json.load(fp)

In [ ]:
def revarnish_link(link_obj):
    if USE_VARNISH:
        for ref in link_obj.refs:
            invalidate_ref(Ref(ref), purge=True)

In [ ]:
def create_link_saver(uid):
    def save_link(link_dict):
        try:
            link_obj = tracker.add(uid, Link, link_dict)
            success = True
        except DuplicateRecordError as e:
            success = False
            print(e)
        if USE_VARNISH and success:
            try:
                revarnish_link(link_obj)
            except Exception as e:
                print(e)
    return save_link

Set `uid` to something normal. Preferrabley your **actual** user id.

In [ ]:
uid = 9e9
link_saver = create_link_saver(uid)

In [ ]:
for i, l in enumerate(links, 1):
    print(f'{i}/{len(links)}')
    link_saver(l)

### Importing Versions

For uploading `Versions` I like to use the backend functions which are used to upload csv files. Use the _download csv_ option on whichever server the version you are deploying to obtain the relevant csv. Then upload said csv to the destination server through kubernetes:

`kubectl cp <filename> <pod>:<path>`

We can then run the following snippet on the destination server backend:

In [ ]:
from sefaria.export import import_versions_from_stream

with open(filename, ‘rb’) as fp:
    import_versions_from_stream(fp, [1], <user_id>)